In [ ]:
import csv
import pandas as pd
import os
import scipy.stats
import numpy as np
from datetime import date,timedelta,datetime

In [ ]:
def read_data(file):
    df = pd.read_csv(file)
    df = pd.DataFrame(df)
    return df

In [ ]:
def mofunc(row):
    if row['Severity'] > 0.8 or row['Hazard_Score'] > 80:
        return 'Warning'
    elif 0.6 < row['Severity'] < 0.80 or 60 < row['Hazard_Score'] < 80:
        return 'Watch'
    elif 0.35 < row['Severity'] < 0.6 or 35 < row['Hazard_Score'] < 60:
        return 'Advisory'
    else:
        return 'Information'

In [ ]:
forcast_date = date.today()
cur_year, cur_month,cur_day = map(str,[forcast_date.today().year,forcast_date.today().month,forcast_date.today().day])
cur_month = cur_month.zfill(2)
cur_day=cur_day.zfill(2)
MOMOutput='Final_Attributes_'+cur_year+cur_month+str(float(cur_day)-1)+'18.csv'
DFO="DFO_"+cur_year+cur_month+str(float(cur_day)-1)+'.csv'

In [ ]:
weightage = read_data('weightage_DFO.csv')
Attributes=read_data('Attributes.csv')
PDC_resilience = read_data('Copy of Resilience_Index.csv')
add_field_DFO=['DFO_area_1day_score', 'DFO_percarea_1day_score', 'DFO_area_2day_score', 'DFO_percarea_2day_score','DFO_area_3day_score', 'DFO_percarea_3day_score','DFOTotal_Score']

In [ ]:
#Read DFO Processing data and calculate score
with open(DFO, 'r', encoding='UTF-8') as DFO_file:
    DFO_reader = csv.reader(DFO_file)
    csvfile = open('DFO_w_score.csv', 'w', newline='\n', encoding='utf-8')
    DFO_w_score = csv.writer(csvfile)
    row_count = 1
    # csv_writer = csv.writer(write_obj)
    for row in DFO_reader:
        if row_count == 1:
            for x in add_field_DFO:
                row.append(x)
            row_count = row_count + 1
        else:
            if float(row[4]) / float(weightage.DFO_Area_wt) > float(weightage.DFO_Area_max_pt):
                DFO_area_1day_score = str(float(weightage.DFO_Area_max_pt)*float(weightage.one_Day_Multiplier))
            else:
                DFO_area_1day_score = str(float(weightage.DFO_Area_Min_pt) * float(weightage.one_Day_Multiplier)* float(row[4]) / float(weightage.DFO_Area_wt))
            if float(row[5]) / float(weightage.DFO_percArea_wt) > float(weightage.DFO_percArea_Maxpt):
                DFO_perc_area_1day_score = str(float(weightage.DFO_percArea_Maxpt)*float(weightage.one_Day_Multiplier))
            else:
                DFO_perc_area_1day_score = str(float(weightage.DFO_percArea_Minpt)*float(weightage.one_Day_Multiplier)* float(row[5]) / float(weightage.DFO_percArea_wt))
            if float(row[6]) / float(weightage.DFO_Area_wt) > float(weightage.DFO_Area_max_pt):
                DFO_area_2day_score = str(float(weightage.DFO_Area_max_pt)*float(weightage.two_Day_Multiplier))
            else:
                DFO_area_2day_score = str(float(weightage.DFO_Area_Min_pt) * float(weightage.two_Day_Multiplier)* float(row[6]) / float(weightage.DFO_Area_wt))
            if float(row[7]) / float(weightage.DFO_percArea_wt) > float(weightage.DFO_percArea_Maxpt):
                DFO_perc_area_2day_score = str(float(weightage.DFO_percArea_Maxpt)*float(weightage.two_Day_Multiplier))
            else:
                DFO_perc_area_2day_score = str(float(weightage.DFO_percArea_Minpt)*float(weightage.two_Day_Multiplier)* float(row[7]) / float(weightage.DFO_percArea_wt))
            if float(row[8]) / float(weightage.DFO_Area_wt) > float(weightage.DFO_Area_max_pt):
                DFO_area_3day_score = str(float(weightage.DFO_Area_max_pt)*float(weightage.three_Day_Multiplier))
            else:
                DFO_area_3day_score = str(float(weightage.DFO_Area_Min_pt) * float(weightage.three_Day_Multiplier)* float(row[8]) / float(weightage.DFO_Area_wt))
            if float(row[9]) / float(weightage.DFO_percArea_wt) > float(weightage.DFO_percArea_Maxpt):
                DFO_perc_area_3day_score = str(float(weightage.DFO_percArea_Maxpt)*float(weightage.three_Day_Multiplier))
            else:
                DFO_perc_area_3day_score = str(float(weightage.DFO_percArea_Minpt)*float(weightage.three_Day_Multiplier)* float(row[9]) / float(weightage.DFO_percArea_wt))
                                          
            Sum_Score = str(
                (float(DFO_area_1day_score) + float(DFO_perc_area_1day_score) + float(DFO_area_2day_score) + float(DFO_perc_area_2day_score)+float(DFO_area_3day_score) + float(DFO_perc_area_3day_score)))
            score_field = [DFO_area_1day_score, DFO_perc_area_1day_score, DFO_area_2day_score, DFO_perc_area_2day_score, DFO_area_3day_score, DFO_perc_area_3day_score,Sum_Score]
            for x in score_field:
                row.append(x)
        DFO_w_score.writerow(row)
csvfile.close()

In [ ]:
DFO = read_data('DFO_w_score.csv')
DFO = DFO[DFO.DFOTotal_Score > 0.1]
DFO = DFO.iloc[:,1:]
MOM = read_data(MOMOutput)
MOM.drop(columns=['area_km2','ISO','Admin0','Admin1','rfr_score','cfr_score','Resilience_Index',' NormalizedLackofResilience ','Severity','Alert'], inplace=True)
Final_Output_0= pd.merge(MOM.set_index('pfaf_id'), DFO.set_index('pfaf_id'), on='pfaf_id', how='outer')
join1 = pd.merge(Attributes, PDC_resilience[['ISO', 'Resilience_Index', ' NormalizedLackofResilience ']], on='ISO', how='inner')
Final_Output=pd.merge(join1.set_index('pfaf_id'), Final_Output_0, on='pfaf_id', how='outer')
Final_Output[['Hazard_Score']] = Final_Output[['Hazard_Score']].fillna(value=0)
Final_Output.loc[(Final_Output['Hazard_Score']!=0) & (Final_Output['Hazard_Score']<Final_Output['DFOTotal_Score']),'Flag']=2
Final_Output['Hazard_Score'] =Final_Output[['Hazard_Score', 'DFOTotal_Score']].max(axis=1)
Final_Output = Final_Output[Final_Output.Hazard_Score != 0]
Final_Output = Final_Output.assign(
    Scaled_Riverine_Risk=lambda x: Final_Output['rfr_score'] * 20)
Final_Output = Final_Output.assign(
    Scaled_Coastal_Risk=lambda x: Final_Output['cfr_score'] * 20)
Final_Output = Final_Output.assign(
    Severity=lambda x: scipy.stats.norm(np.log(100 - Final_Output[['Scaled_Riverine_Risk', 'Scaled_Coastal_Risk']].max(axis=1)), 1).cdf(
        np.log(Final_Output['Hazard_Score'])))
Final_Output['Alert'] = Final_Output.apply(mofunc, axis=1)
Final_Output.to_csv('Final_Attributes_'+cur_year+cur_month+str(float(cur_day)-1)+'18_DFOUpdated.csv', encoding='utf-8-sig')
join1 = pd.merge(Attributes, PDC_resilience[['ISO', 'Resilience_Index', ' NormalizedLackofResilience ']], on='ISO', how='inner')
Attributes_Clean_DFO_Updated = pd.merge(join1.set_index('pfaf_id'), Final_Output[['Alert','Flag']], on='pfaf_id', how='right')
Attributes_Clean_DFO_Updated.to_csv('Attributes_Clean'+cur_year+cur_month+str(float(cur_day)-1)+'18_DFOUpdated.csv', encoding='utf-8-sig')
os.remove('DFO_w_score.csv')